### AE approach
---
The AE approach tries to let an AE look at scenes where a given quantifier q was used by a teacher (language speaker), this is repeated many times till whatever structure typical to scenes True under the q quantifier are encoded in the AE structure and each representation of this structure is given in the hidden vector. When learning is done and given a scene we use the AE as an anomaly classifier to decide whether the scene is True under the q quantifier. The idea is that after seeing many q True scenes a non q True scene will have relatively high reconstruction errors.

Many types of AEs can be used and are implemented

## Imports

### my class imports

In [15]:
from Q.quants import *
from Q.models import AE

Using TensorFlow backend.


ImportError: cannot import name 'AE' from 'Q.models' (/home/doron/git/research/notebooks/quantifiers/Q/models.py)

### Global Imports

In [1]:
import numpy as np
import pandas as pd

### keras imports

In [37]:
# keras imports

import tensorflow as tf

print("TensorFlow version: ", tf.__version__)

import keras

from keras.models import Sequential, Model
from keras.layers import SimpleRNN, LSTM, Dense, Conv1D, MaxPooling1D, UpSampling1D, Input, Bidirectional, RepeatVector, TimeDistributed, Dropout, LeakyReLU, Flatten
from keras.utils import plot_model


TensorFlow version:  2.2.0


Using TensorFlow backend.


In [40]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU'),
 LogicalDevice(name='/device:XLA_GPU:0', device_type='XLA_GPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [41]:
# from functools import partial, update_wrapper

# def wrapped_partial(func, *args, **kwargs):
#         partial_func = partial(func, *args, **kwargs)
#         update_wrapper(partial_func, func)
#         return partial_func

### AE Models

In [42]:
def DenseAE():
    model = Sequential()
    #encoding
    model.add(Dense(75, activation='sigmoid', input_shape=(100, 3)))
    model.add(Flatten())
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dense(25, activation='sigmoid'))
    #decoding
    model.add(Dense(50, activation='sigmoid'))
    model.add(Dense(75, activation='sigmoid'))
    model.add(Dense(Quantifier.scenes_len, activation='sigmoid'))
    model.compile(optimizer='adam', loss='mse')
    return model

def CNNAE():
    model= Sequential()
    #encoding
    model.add(Conv1D(60,32, strides=1, activation='relu',padding='causal',input_shape=(Quantifier.scenes_len,1)))
    model.add(Conv1D(80,10, strides=1, activation='relu',padding='causal'))
    model.add(Dropout(0.25))
    model.add(Conv1D(100,5, strides=1, activation='relu',padding='causal'))
    model.add(MaxPooling1D(1))
    #decoding
    model.add(Dropout(0.25))
    model.add(Dense(300,activation='relu'))
    model.add(Dense(1,activation='relu'))
    model.compile(optimizer='adam', loss='mse')
    return model

def RNNAE():
    model = Sequential()
    model.add(Bidirectional(SimpleRNN(15, activation='sigmoid', input_shape=(Quantifier.scenes_len, 1))))
    model.add(Dropout(0.5))
#     model.add(Bidirectional(SimpleRNN(15, activation='sigmoid', return_sequences=True)))
#     model.add(Dropout(0.5))
    model.add(RepeatVector(Quantifier.scenes_len))
    model.add(SimpleRNN(15, activation='sigmoid', return_sequences=True))
    model.add(Dropout(0.5))
#     model.add(Bidirectional(SimpleRNN(25, activation='sigmoid', return_sequences=True)))
#     model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    return model

def LSTMAE():
    model = Sequential()
    model.add(LSTM(15, activation='sigmoid', input_shape=(Quantifier.scenes_len, 1)))
    model.add(Dropout(0.5))
    model.add(RepeatVector(Quantifier.scenes_len))
    model.add(LSTM(15, activation='sigmoid', return_sequences=True))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    return model

def BLSTMAE():
    model = Sequential()
    model.add(Bidirectional(LSTM(15, activation='sigmoid', input_shape=(Quantifier.scenes_len, 1))))
    model.add(Dropout(0.5))
    model.add(RepeatVector(Quantifier.scenes_len))
    model.add(Bidirectional(LSTM(15, activation='sigmoid', return_sequences=True)))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(1)))
    model.compile(optimizer='adam', loss='mse')
    return model

### Learn

In [56]:
num_epochs = 1
batch_size = 10

def learn(scenes):
#     model = DenseAE()
#     model = CNNAE()
#     model = RNNAE()
#     model = LSTMAE()
    model = BLSTMAE()
    plot_model(model, to_file='AE_model.png', show_shapes=True)
#     model.fit(Quantifier.one_hot(scenes), scenes, batch_size=batch_size, epochs=num_epochs, verbose=1)
    model.fit(scenes, scenes, batch_size=batch_size, epochs=num_epochs, verbose=1)
    return model

### Test

In [57]:
def test_quantifer(model, quantifier):
    test_size = 10

    test_scenes = generate_random_scenes(scenes_num=test_size)
    test_labels = [quantifier.quantify(test_scene) for test_scene in test_scenes]
    test_results = [model.evaluate(test_scene.reshape(1, -1, 1), 
                                   test_scene.reshape(1, -1, 1), verbose=0) for test_scene in test_scenes]
    thresh = np.mean(test_results)
    test_thresh = [test_result < thresh for test_result in test_results]
    print(np.sum(np.array(test_labels) == np.array(test_thresh)))

In [58]:
quantifiers = [Between_3_6, Between_2_10, One, Two, Three, Ten, Some, Few, No, Every, Most]

def test_quantifers(model):
    test_size = 1000
    test_results = pd.DataFrame(columns=[quantifier.__name__ for quantifier in quantifiers])

#     for i in range(test_size):
    test_result = []
    for quantifier in quantifiers:
        test_scenes = generate_quantified_scenes(quantifier, scenes_num=test_size)
        test_result.append(model.evaluate(test_scenes, test_scenes, verbose=0))

    test_results.loc[0] = test_result
    return test_results    

In [59]:
# with tf.device("/cpu:0"):
with tf.device("/gpu:0"):
    # fit model
    model = learn(generate_quantified_scenes(Every))
#     model = learn(generate_quantified_scenes(Every))


Epoch 1/1
1000/1000 [==============================] - 30s 30ms/step - loss: 0.8308


In [60]:
with tf.device("/gpu:0"):
    test_quantifer(model, Every)
#     models = [learn(generate_quantified_scenes(quantifier)) for quantifier in quantifiers]

8
